In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import csv
import pandas as pd
import numpy as np
import time

import os


from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.pipeline.core import Pipeline, ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep

from azureml.core import Workspace, Dataset

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

from azureml.core import Workspace, Dataset, Datastore

workspace = Workspace.from_config()

datastore = workspace.get_default_datastore()

In [3]:
environment = Environment.from_conda_specification("nfl_env", "nfl_env.yaml")

In [4]:
# Create a compute target
compute_name = "matthewhanauer991"
# Define the environment and dependencies for your script


try:
    compute_target = ComputeTarget(workspace=workspace, name=compute_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [32]:
import os

script_directory = os.path.dirname(os.path.abspath("production_injury_data.py"))
script_file = "production_injury_data.py"

src = ScriptRunConfig(
    source_directory=script_directory, script=script_file, environment=environment
)

In [33]:
step = PythonScriptStep(
    name="scrapper_nfl_injury",
    script_name=script_file,
    compute_target=compute_target,
    runconfig=src.run_config,
    allow_reuse=False,
)

In [34]:
# Create a Pipeline using the PythonScriptStep
pipeline = Pipeline(workspace=workspace, steps=[step])

In [35]:
# Publish the pipeline
published_pipeline = pipeline.publish(name="scrapper_nfl_injury_pipeline")

Created step scrapper_nfl_injury [3e779916][db11ea06-533f-46f2-9b7e-6e7785daea4e], (This step will run and generate new outputs)


In [37]:
pipeline_run = Experiment(workspace, "scrapper_nfl_injury_experiment").submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Submitted PipelineRun 9d9c76ac-480b-498e-b39f-e03b6a4b1067
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/9d9c76ac-480b-498e-b39f-e03b6a4b1067?wsid=/subscriptions/d3ade6e0-a7be-4323-ace4-02b7b4da2451/resourcegroups/ml_modeling/workspaces/mhanauer&tid=a93eafc8-c064-4a88-bc01-47e93068b42b
PipelineRunId: 9d9c76ac-480b-498e-b39f-e03b6a4b1067
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/9d9c76ac-480b-498e-b39f-e03b6a4b1067?wsid=/subscriptions/d3ade6e0-a7be-4323-ace4-02b7b4da2451/resourcegroups/ml_modeling/workspaces/mhanauer&tid=a93eafc8-c064-4a88-bc01-47e93068b42b
PipelineRun Status: NotStarted
PipelineRun Status: Running




PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '9d9c76ac-480b-498e-b39f-e03b6a4b1067', 'status': 'Completed', 'startTimeUtc': '2023-05-16T18:51:28.144195Z', 'endTimeUtc': '2023-05-16T18:52:05.3681Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK

Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


'Finished'

Step: Schedule the recurrence manually by looking at the job and scheduling